Установка необходимых библиотек

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

classifier_dataset	dataset      drive     recommendations.csv
classifier_dataset.zip	dataset.zip  __MACOSX  sample_data


In [4]:
!cp /content/drive/MyDrive/project/dataset.zip /content/

In [ ]:
!cp /content/drive/MyDrive/project/recommendations.csv /content/

In [ ]:
!cp /content/drive/MyDrive/project/classifier_dataset.zip /content/

In [5]:
!pip install pytesseract

In [6]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [7]:
!sudo apt install tesseract-ocr-rus

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr-rus is already the newest version (1:4.00~git30-7274cfa-1.1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [8]:
!pip install datefinder

In [ ]:
!unzip dataset.zip

Archive:  dataset.zip
replace __MACOSX/._dataset? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!unzip classifier_dataset.zip

Archive:  classifier_dataset.zip
replace classifier_dataset/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Создание класса датасета.

В данном классе происходят трансформации изображения и поиск всех областей с текстом.

In [9]:
# все трансформации
import cv2

class Dilation(object):
  def __init__(self, kernel_size=20, iterations=1):
    self.kernel_size = kernel_size
    self.iterations = iterations

  def __call__(self, img):
    ret, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (self.kernel_size, self.kernel_size))
    dilation = cv2.dilate(thresh, kernel, iterations = self.iterations)
    return dilation

class ToGrayScale(object):
  def __init__(self):
    pass

  def __call__(self, img):
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return grayscale

In [10]:
from glob import glob
from torchvision.transforms import Compose

class FoodDataset:
  def __init__(self, root_dir, transformations=None):
    self.root_dir = root_dir
    self.imgs = sorted(glob(self.root_dir + "/*/*"))
    self.transformations = None
    if transformations:
      self.transformations = Compose(transformations)

  def __len__(self):
    return len(self.imgs)

  def __getitem__(self, idx):
    img_path = self.imgs[idx]
    im = cv2.imread(img_path)
    category = img_path.split('/')[-2]

    # сначала трансформируем изображение
    if self.transformations:
      transformed_img = self.transformations(im)

    # затем достаем кусочки изображения с текстом
    imgs, dilations = self._get_single_images(transformed_img, im)

    return img_path, im, imgs, dilations, category

  def _get_single_images(self, dilation, image):
    contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,
                                                 cv2.CHAIN_APPROX_NONE)
    images = []
    dilations = []

    for contour in contours:
      img = image.copy()
      dil = dilation.copy()
      x, y, w, h = cv2.boundingRect(contour)
      cropped_img = img[y:y + h, x:x + w]
      cropped_dilation = dil[y:y+h, x:x+w]
      dilations.append(cropped_dilation)
      images.append(cropped_img)

    return images, dilations

In [11]:
import pytesseract
import datefinder
from PIL import Image
import datetime

pytesseract.pytesseract.tesseract_cmd = (
    r'/usr/bin/tesseract'
)

def extract_dates(texts):
  dates = []
  for text in texts:
    text = ''.join(text.split())
    found_dates = list(datefinder.find_dates(text))
    dates.extend(found_dates)
  dates = list(set(dates))
  return dates

def get_all_texts(im, imgs, dilations):
  texts = [pytesseract.image_to_string(Image.fromarray(im[0].numpy()), lang="rus")]
  for image in imgs:
    txt = pytesseract.image_to_string(Image.fromarray(image[0].numpy()), lang="rus")
    texts.append(txt)
  for dil in dilations:
    txt = pytesseract.image_to_string(Image.fromarray(dil[0].numpy()), lang="rus")
    texts.append(txt)

  return texts

def make_recommendations(df, dates, category):
  if len(dates) == 0:
    print("Ничего неизвестно о дате выпуска продукта.")
    return
  product_info = df.loc[df['class'] == category]
  current_date = datetime.datetime.now()
  manufac_date = min(dates)
  expire_date = max(dates)
  if manufac_date == expire_date:
    expire_date += datetime.timedelta(days=int(product_info['avg_shelf_life']))
  shelf_life = (expire_date - manufac_date).days
  recommended = float(product_info['recommended'])
  safe_period = float(product_info['safe_period'])
  if shelf_life * recommended < (expire_date - current_date).days:
    print("Продукт свежий, рекомендован к употреблению")
    return
  if recommended < 1:
    if manufac_date + datetime.timedelta(days=shelf_life) > current_date:
      print("Срок годности не истек, но подходит к концу. Не рекомендован людям с индивидуальной непереносимостью")
    elif manufac_date + datetime.timedelta(days=shelf_life) == current_date:
      print("Срок годности истекает сегодня. Не рекомендовано к употреблению людям с индивидуальной непереносимостью")
    else:
      print("Срок годности истек. Употреблять нельзя")
  else:
    if manufac_date + datetime.timedelta(days=shelf_life + safe_period) <= current_date:
      print("Срок годности истек, однако продукт еще возможно употреблять людям без заболеваний")
    else:
      print("Срок годности и запас срока годности истек. Употреблять нельзя")

In [12]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd

transformations = [ToGrayScale(), Dilation()]
classifier_dataset = FoodDataset('classifier_dataset', transformations)
classifier_dataloader = DataLoader(classifier_dataset, batch_size=1, shuffle=False)

texts = []
labels = []

for path, im, imgs, dilations, category in tqdm(classifier_dataloader):
    texts.append(' '.join(get_all_texts(im, imgs, dilations)))
    labels.append(category)

100%|██████████| 47/47 [20:07<00:00, 25.69s/it]


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline

model = make_pipeline(TfidfVectorizer(), MultinomialNB())
model.fit(texts, labels)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('tfidfvectorizer', TfidfVectorizer()),
                ('multinomialnb', MultinomialNB())])

In [35]:
from sklearn.metrics import accuracy_score, log_loss

predicted_labels = model.predict(texts)
print("Accuracy: ", accuracy_score(labels, predicted_labels))

ValueError: Found input variables with inconsistent numbers of samples: [47, 71]

In [ ]:
print("Logloss: ", log_loss(labels, predicted_labels))

In [33]:
labels

[('cheese',),
 ('cheese',),
 ('cheese',),
 ('cheese',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('milk',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('pasta',),
 ('proper_nutrition',),
 ('proper_nutrition',),
 ('proper_nutrition',),
 ('proper_nutrition',),
 ('proper_nutrition',),
 ('protein',),
 ('protein',),
 ('protein',),
 ('protein',),
 ('protein',),
 ('protein',),
 ('protein',),
 ('sweets',),
 ('sweets',),
 ('sweets',),
 ('sweets',),
 ('sweets',),
 ('sweets',),
 ('tea',),
 ('tea',),
 ('tea',),
 ('tea',)]

In [ ]:
print(predicted_labels)

In [22]:
texts[5]

'     \n\n       \n \n   \n\nой з\nны п о\n\nПОСЛЕ УПОТРЕБЛЕНИЯ Дата производства:\nОСТАВЬ ТРУБОЧКУ ВНУТРИ!\nРР. 2 : — 75\n\n20.03.24 15\n\nУпотребить до:\n\x0c \x0c \x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c \x0c \x0c  \n\x0c  \n\x0c \x0c \x0c \x0c  \n\x0c \x0c \x0c  \n\x0c \x0c \x0c  \n\x0c \x0c  \n\x0c \x0c \x0c \x0c \x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c  \n\x0c \x0c  \n\x0c  \n\x0c \x0c  \n\x0c \x0c \x0c \x0c \x0c  \n\x0c \x0c 2:Н .\n20.03.\n\nУпотребить до:\n\x0c \x0c  \n\x0c \x0c ПОСЛЕ УПОТРЕБЛЕНИЯ\nОСТАВЬ ТРУБОЧКУ ВНУТРИ!\n\x0c Дата производства:\n\x0c  \n\x0c \x0c  \n\x0c \x0c \x0c \x0c \x0c \x0c \x0c          \n  \n\n>\n\n \n\nПОСЛЕ УПОТРЕБЛЕНИЯ Дата производства:\n\nОСТАВЬ ТРУБОЧКУ ВНУТРИ!\n21.11.23 08\nЯ\n\n20.03.24 15\n\nУпотребить до:\n\n    \n     \n      \n\x0c \x0c \x0c \x0c \x0c  \n\x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c  \n\x0c  \n\x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c \x0c  \n\x0c \x0c \x0c \x0c  \n\x0c \x0c \x0c \x0c \x0c \

In [24]:
model.predict([texts[5]])[0]

'milk'

In [25]:
transformations = [ToGrayScale(), Dilation()]
dataset = FoodDataset('dataset', transformations)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
recommends = pd.read_csv('recommendations.csv', sep=";")

for path, im, imgs, dilations, _ in tqdm(dataloader):
  test_texts = get_all_texts(im, imgs, dilations)
  category = model.predict([' '.join(test_texts)])[0]
  dates = extract_dates(test_texts)
  print("Изображение: " + path[0])
  make_recommendations(recommends, dates, category)

  2%|▏         | 1/55 [00:20<18:34, 20.63s/it]

Изображение: dataset/cheese/1.JPG
Срок годности не истек, но подходит к концу. Не рекомендован людям с индивидуальной непереносимостью


  4%|▎         | 2/55 [00:42<18:57, 21.47s/it]

Изображение: dataset/cheese/10.JPG
Продукт свежий, рекомендован к употреблению


  5%|▌         | 3/55 [01:09<20:47, 23.99s/it]

...

Изображение: dataset/tea/2.JPG
Ничего неизвестно о дате выпуска продукта.


 98%|█████████▊| 54/55 [25:01<00:34, 34.30s/it]

Изображение: dataset/tea/3.JPG
Продукт свежий, рекомендован к употреблению


100%|██████████| 55/55 [25:36<00:00, 27.94s/it]

Изображение: dataset/tea/4.JPG
Срок годности истек. Употреблять нельзя
